In [1]:

#巡回セールスマン問題を蟻コロニー最適化と遺伝的アルゴリズムで解いてみる[Python]---棚移動距離で距離行列を作成
#http://tony-mooori.blogspot.com/2016/02/tsppython.html

#始点と終点をインプットデータのままにする

#coding:utf-8
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance as dis


In [2]:

#勤務地設定--------
#os.chdir(r"D:\_User\_SqlLocalDb")
os.chdir(r"/Users/machioka_masaki/Downloads/20181001_SatoResult/20180911_SatoSimuTrial004")
print("ワーキングディレクトリは以下のです")
print(os.getcwd() )




ワーキングディレクトリは以下のです
/Users/machioka_masaki/Downloads/20181001_SatoResult/20180911_SatoSimuTrial004


In [3]:

"""
参考URL
[1] 蟻コロニー最適化 - Wikipedia https://ja.wikipedia.org/wiki/蟻コロニー最適化
[2] 任意の確率密度分布に従う乱数の生成（von Neumannの棄却法） | Pacocat's Life http://pacocat.com/?p=596
"""


#ピッキングオーダー表の取り込み
PikOrder000 = pd.read_csv("TanaGrid.csv")
PikOrder000

,Tana,x,y
0,B90275,31,34
1,B50524,28,29
2,AL2011,8,21
3,F51031,52,33
4,F60514,52,28
5,G81121,54,10
6,J26621,82,4
7,J35131,84,20
8,JE1021,104,11
9,NJ1711,152,69


In [4]:
#距離行列計算


#巡回ポイントロギング用マトリックス作成
#df0mat = ( PikOrder000['x'].astype(str) + '-'  + PikOrder000['y'].astype(str) ).as_matrix()
df0mat = ( PikOrder000['x'].astype(str) + '-'  + PikOrder000['y'].astype(str) ).to_numpy() 

df0mat

#重複を削除
df4 = PikOrder000.drop_duplicates(['x','y']).copy()
df4.reset_index()
df5 = df4[["x","y"]]

locations=np.array(df5)
locs=locations
""" 位置座標を設定する関数 """
self_loc = locs                            # x,y座標
self_n_data = len(self_loc)                        # データ数

#X軸移動距離

self_locX0 = np.copy(self_loc) # X軸距離算出用配列を複製生成
self_locX0[:,1]=0 #Y軸座標をゼロ置換
self_distX0 = dis.cdist(self_locX0,self_locX0,metric='euclidean')    # 距離の表を作成2---X軸移動距離



#Y軸移動距離
self_locY0 = np.copy(self_loc) # Y軸距離算出用配列を複製生成--その1--同一列用
self_locY0[:,0]=0 #X軸座標をゼロ置換
self_distY0 = dis.cdist(self_locY0,self_locY0,metric='euclidean')    # 距離の表を作成2---X軸移動距離


##### X軸距離と（列相違調整後の）Y軸距離の合計が総距離 ####
self_dist0 = self_distX0 + self_distY0

####self_dist_aco --- 
self_dist_aco0 = self_dist0


In [5]:
#pd.DataFrame(self_distX0).to_csv("self_distX0.csv")
#pd.DataFrame(self_distY0).to_csv("self_distY0.csv")

In [6]:
#取り込むcsvは1行目は行名ヘッダーとするが1列目に列名は付けない
self_distX1 = pd.read_csv("self_distX0Edited.csv").values
self_distY1 = pd.read_csv("self_distY0Edited.csv").values



In [7]:
type(self_distX1)

numpy.ndarray

In [8]:
###繰り返し試行回数
Kaisu = 100

###巡回箇所の数
#junkaisu = 3

###アリさんの数
n_agent=1000

#行列番地の最大値
maxX = 157 #East方式用on小林さんマップ
maxY = 72


In [9]:
#結果出力用ファイルの時間名称用変数
import time
timestr = time.strftime("%Y%m%d_%H%M%S")



for i in range( Kaisu ):


    #class TSP:
    #def __init__(self,path=None,alpha = 1.0,beta = 1.0,Q = 1.0,vanish_ratio = 0.95):
    alpha = 1.0
    beta = 1.0
    Q = 1.0
    vanish_ratio = 0.95

    """ 初期化を行う関数 """
    self_alpha = alpha                    # フェロモンの優先度
    self_beta = beta                    # ヒューリスティック情報(距離)の優先度
    self_Q = Q                            # フェロモン変化量の係数
    self_vanish_ratio = vanish_ratio    # 蒸発率

    #locations=np.array(df5)

    #距離行列データ取り込み
    ##### X軸距離と（列相違調整後の）Y軸距離の合計が総距離 ####
    self_dist = self_distX1 + self_distY1
    ####self_dist_aco --- 
    self_dist_aco = self_dist

    #フェロモンと順序保存
    self_weight = np.random.random_sample((self_n_data,self_n_data))    # フェロモンの量
    self_result = np.arange(self_n_data)            # もっともよかった順序を保存する

    #内容参照用---省略


    #def solve(self,n_agent=1000):
    """ 巡回セールスマン問題を蟻コロニー最適化で解く """

    #order = np.zeros(self_n_data,np.int)         # 巡回経路
    order = np.zeros(self_n_data,np.int64)         # 巡回経路
    delta = np.zeros((self_n_data,self_n_data))    #フェロモン変化量

    #n_agent=1000
    for k in range(n_agent):
        city = np.arange(self_n_data)
        #now_city = np.random.randint(self_n_data)    # 現在居る都市番号
        now_city = 0 # 最初の都市番号を0として始点x1y1に定める
        #print(now_city)
        last_city = self_n_data - 1

        city = city[ city != now_city ]
        city = city[ city != last_city ]
        order[0] = now_city
        order[last_city] = last_city

        for j in range(1,self_n_data-1): #rangeの「self_n_data-1」迄としたのは、rangeの最後に終点が格納されている為

            upper = np.power(self_weight[now_city,city],self_alpha)*np.power(self_dist_aco[now_city,city],-self_beta)

            evaluation = upper / np.sum(upper)                # 評価関数
            percentage = evaluation / np.sum(evaluation)    # 移動確率
            #index = self_random_index(percentage)            # 移動先の要素番号取得
            n_percentage = len(percentage)

            while True:
                index = np.random.randint(n_percentage)
                y = np.random.random()
                if y < percentage[index]:
                    #return index        
                    break
            #print(index)
            # 状態の更新
            now_city = city[ index ]
            city = city[ city != now_city ]
            order[j] = now_city


        #L = self_cost(order) # 経路のコストを計算
        n_order = len(order)
        #L = np.sum( [ self_dist_aco[order[i],order[(i+1)%n_order]] for i in np.arange(n_order) ] )
        L = np.sum( [ (self_dist_aco[order[i],order[(i+1)]])  for i in np.arange(n_order-1) ] )

        # フェロモンの変化量を計算
        delta[:,:] = 0.0
        c = self_Q / L
        for j in range(self_n_data-1):
            delta[order[j],order[j+1]] = c
            delta[order[j+1],order[j]] = c

        # フェロモン更新
        self_weight *= self_vanish_ratio 
        self_weight += delta


        #print(self_result)
        # 今までで最も良ければ結果を更新
        n_order2 = len(self_result)
        #L2 =  np.sum( [ self_dist_aco[self_result[i],self_result[(i+1)%n_order2]] for i in np.arange(n_order2) ] )
        L2 = np.sum( [ (self_dist_aco[self_result[i],self_result[(i+1)]])  for i in np.arange(n_order2-1) ] )

        if L2 > L:
            self_result = order.copy()

    #https://qiita.com/a_t_a/items/50623aedf7e1dab3b876
    import csv
    with open(timestr + '_result.csv','a',newline='') as f:
        writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
        # ヘッダー
        #header = ['Route', 'Distance']
        #writer.writerow(header)
        # データをリストに保持
        csvlist = []
        csvlist.append( df0mat )
        csvlist.append(self_result)
        csvlist.append(  L2  ) #幹距離
        writer.writerow(csvlist)
        f.close()
